In [12]:
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os

In [13]:
df = pd.read_csv('../../../../Data/housing_data_encoded.csv',infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="State_TX")
df = df.sort_index()
df

,Date,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
0,2010-01-31,2,1,67653.0,133.0,0,1,0,0,1,1,0
1,2010-01-31,2,2,89995.0,133.0,0,1,0,0,1,1,0
2,2010-01-31,2,3,116641.0,133.0,0,1,0,0,1,1,0
3,2010-01-31,2,4,167849.0,133.0,0,1,0,0,1,1,0
4,2010-01-31,2,5,303323.0,133.0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1215,2020-02-29,223,1,79380.0,100.0,0,0,1,0,1,1,0
1216,2020-02-29,223,2,121517.0,100.0,0,0,1,0,1,1,0
1217,2020-02-29,223,3,159204.0,100.0,0,0,1,0,1,1,0
1218,2020-02-29,1811,2,157660.0,108.0,0,0,0,1,1,0,1


In [14]:
#Citation: https://stackoverflow.com/questions/53731432/return-pandas-multiindex-as-list-of-tuples

arrays = [df["Identifier"].replace(" ",""),
          df["Bedroom"].replace(" ",""),
          df["Date"].replace(" ","")]

tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['Region_ID', 'Bedrooms','Date'])

TX_nbhoods_df = df.set_index(index)
TX_nbhoods_df = TX_nbhoods_df.drop(columns=["Identifier","Bedroom","Date"])
TX_nbhoods_df = TX_nbhoods_df.sort_index()
TX_nbhoods_df

Price  Average_Market_Days  \
Region_ID Bedrooms Date                                        
2         1        2010-01-31   67653.0                133.0   
                   2010-02-28   67444.0                137.0   
                   2010-03-31   67260.0                134.0   
                   2010-04-30   67041.0                112.0   
                   2010-05-31   66721.0                 92.5   
...                                 ...                  ...   
2134      2        2019-10-31  126817.0                140.0   
                   2019-11-30  126739.0                142.0   
                   2019-12-31  126353.0                185.0   
                   2020-01-31  125649.0                145.0   
                   2020-02-29  125585.0                156.0   

                               RegionName_Crosby  RegionName_Houston  \
Region_ID Bedrooms Date                                                
2         1        2010-01-31                  0                   1   
                   2010-02-28                  0                   1   
                   2010-03-31                  0                   1   
                   2010-04-30                  0                   1   
                   2010-05-31                  0                   1   
...                                          ...                 ...   
2134      2        2019-10-31                  1                   0   
                   2019-11-30                  1                   0   
                   2019-12-31                  1                   0   
                   2020-01-31                  1                   0   
                   2020-02-29                  1                   0   

                               RegionName_Pasadena  RegionName_Porter  \
Region_ID Bedrooms Date                                                 
2         1        2010-01-31                    0                  0   
                   2010-02-28                    0                  0   
                   2010-03-31                    0                  0   
                   2010-04-30                    0                  0   
                   2010-05-31                    0                  0   
...                                            ...                ...   
2134      2        2019-10-31                    0                  0   
                   2019-11-30                    0                  0   
                   2019-12-31                    0                  0   
                   2020-01-31                    0                  0   
                   2020-02-29                    0                  0   

                               Metro_Houston-The Woodlands-Sugar Land  \
Region_ID Bedrooms Date                                                 
2         1        2010-01-31                                       1   
                   2010-02-28                                       1   
                   2010-03-31                                       1   
                   2010-04-30                                       1   
                   2010-05-31                                       1   
...                                                               ...   
2134      2        2019-10-31                                       1   
                   2019-11-30                                       1   
                   2019-12-31                                       1   
                   2020-01-31                                       1   
                   2020-02-29                                       1   

                               CountyName_Harris County  \
Region_ID Bedrooms Date                                   
2         1        2010-01-31                         1   
                   2010-02-28                         1   
                   2010-03-31                         1   
                   2010-04-30                         1   
          

In [15]:
TX_nbhoods_df.index

MultiIndex([(   2, 1, '2010-01-31'),
            (   2, 1, '2010-02-28'),
            (   2, 1, '2010-03-31'),
            (   2, 1, '2010-04-30'),
            (   2, 1, '2010-05-31'),
            (   2, 1, '2010-06-30'),
            (   2, 1, '2010-07-31'),
            (   2, 1, '2010-08-31'),
            (   2, 1, '2010-09-30'),
            (   2, 1, '2010-10-31'),
            ...
            (2134, 2, '2019-05-31'),
            (2134, 2, '2019-06-30'),
            (2134, 2, '2019-07-31'),
            (2134, 2, '2019-08-31'),
            (2134, 2, '2019-09-30'),
            (2134, 2, '2019-10-31'),
            (2134, 2, '2019-11-30'),
            (2134, 2, '2019-12-31'),
            (2134, 2, '2020-01-31'),
            (2134, 2, '2020-02-29')],
           names=['Region_ID', 'Bedrooms', 'Date'], length=1220)

In [16]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [17]:
def build_by_region_beds(tuple_,window_size):
    region_bed = tuple_[:2]
    region_bed_df = TX_nbhoods_df.loc[region_bed]
    prices_df = region_bed_df.loc[:,["Price"]]
    prices_df.sort_index(inplace=True)
    feature_column = 0
    target_column = 0
    X, y = window_data(prices_df, window_size, feature_column, target_column)

    split = int(0.7 * len(X))
    X_train = X[: split]
    X_old = prices_df
    X_old.rename(columns={"Price":"Real"})
    X_test = X[split:]
    y_train = y[: split]
    y_test = y[split:]

    scaler = MinMaxScaler()
    scaler.fit(X)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    scaler.fit(y)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model = Sequential()

    number_units = 5
    dropout_fraction = 0.2


    model.add(LSTM(
        units=number_units,
        return_sequences=True,
        input_shape=(X_train.shape[1], 1))
        )
    model.add(Dropout(dropout_fraction))

    model.add(LSTM(units=number_units, return_sequences=True))
    model.add(Dropout(dropout_fraction))

    model.add(LSTM(units=number_units))
    model.add(Dropout(dropout_fraction))

    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mean_squared_error")


    model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

    loss = model.evaluate(X_test, y_test)
    predicted = model.predict(X_test)

    predicted_prices = scaler.inverse_transform(predicted)
    real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

    index= [date for date in X_old.index]
    final_df = pd.DataFrame({
        "Real": real_prices.ravel(),
        "Predicted": predicted_prices.ravel()},
        index= index[-len(real_prices):]) 
    

    return [final_df, loss, X_old]


In [19]:
for tuple_ in set(TX_nbhoods_df.index):
    for i in [1,3,6,12]:
        path = f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/'
        if os.path.isdir(path)==False:
            os.makedirs(path)
            #Citation: https://docs.python.org/3/tutorial/controlflow.html
            continue
        else:
            continue

        file_list = os.listdir(path)
        print(file_list)
        if len(file_list)==4:
            pass
        else:
            model = build_by_region_beds(tuple_,i)
            output = model[0]
            loss_val = model[1]
            old_prices = model[2]
            index = pd.MultiIndex.from_tuples([tuple_[:2]],names=["Region","Bedrooms"])
            #Note to self: If you don't end up figuring out why its predicting Nan's for some of them, remember to drop those region_beds from the data before AWS run (window too large for some datasets?)
            loss = pd.DataFrame(columns={"Loss":loss_val},index=index)
            loss["Loss"] = loss_val
            Predicted_vs_Real = output.plot()
            Train_vs_Test = pd.concat([old_prices, output["Predicted"]],axis=1,join='outer')
            Train_vs_Test_plt = Train_vs_Test.plot()

            output.to_csv(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Results.csv')
            loss.to_csv(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Loss.csv') 

            # Citation: https://stackoverflow.com/questions/18992086/save-a-pandas-series-histogram-plot-to-file
            Predicted_vs_Real.figure.savefig(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Plot.png')
            Train_vs_Test_plt.figure.savefig(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Plot2.png')

In [ ]:
# ## OLD FOR-LOOP IN CASE THE NEW ONE BREAKS:
# for tuple_ in set(TX_nbhoods_df.index):
#     for i in [1,3,6,12]:
#         model = build_by_region_beds(tuple_,i)
#         output = model[0]
#         loss_val = model[1]
#         old_prices = model[2]
#         index = pd.MultiIndex.from_tuples([tuple_[:2]],names=["Region","Bedrooms"])
#         #Note to self: If you don't end up figuring out why its predicting Nan's for some of them, remember to drop those region_beds from the data before AWS run (window too large for some datasets?)
#         loss = pd.DataFrame(columns={"Loss":loss_val},index=index)
#         loss["Loss"] = loss_val
#         Predicted_vs_Real = output.plot()
#         Train_vs_Test = pd.concat([old_prices, output["Predicted"]],axis=1,join='outer')
#         print(Train_vs_Test)
#         Train_vs_Test_plt = Train_vs_Test.plot()

#         path = f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/'
        
        
#         if os.path.isdir(path)==True:
#             pass
            
#         else:
#             os.makedirs(path)
#         output.to_csv(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Results.csv')
#         loss.to_csv(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Loss.csv') 

#         # Citation: https://stackoverflow.com/questions/18992086/save-a-pandas-series-histogram-plot-to-file
#         Predicted_vs_Real.figure.savefig(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Plot.png')
#         Train_vs_Test_plt.figure.savefig(f'../Results/Region/{tuple_[0]}/Bedrooms/{tuple_[1]}/Window-Size/{str(i)}/Plot2.png')